In [1]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import xgboost
import pandas as pd
import numpy as np
import pickle
from sklearn.preprocessing import LabelEncoder
from pathlib import Path
from tqdm.auto import tqdm

tqdm.pandas()

In [2]:

df = pd.read_parquet('C:/Saravana/Projects/Intellizenz/intellizenz-model-training/data/export_features_2016_2020_v2.parquet.gzip')

In [ ]:
bands_df = pd.read_parquet('C:/Saravana/Projects/Intellizenz/intellizenz-model-training/data/export_band_stats_for_range_of_vginkasso_2016_2020.parquet.gzip')
display(bands_df)

In [3]:
pd.set_option('display.max_columns', None)

In [4]:
df=df.rename(columns = {'band_x':'band', 'vg_state_x':'vg_state'})

In [19]:
# Split the data set and extract time series features
X = df.loc[:,~df.columns.isin(['veranst_segment','vg_inkasso'])]
y = df['vg_inkasso']

In [21]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20)

In [5]:
display(df.head(5))

,imp_id,gj,import,mufo_referenz_n,barcode_nr,veranst_segment,rekla_jn,vg_datum_von,vg_ort,vg_raum,nutzfall,nutzfall_raum,musikleiter_name,kapelle_name,tarif_nr,tarif_bez,nutzfall_nr,vg_inkasso,inkasso_netto,inkasso_brutto,veranst_geschaeftszeichen,veranst_name,veranst_strasse,veranst_plz,veranst_ort,nutzliznehm_geschaeftszeichen,nutzliznehm_name,nutzliznehm_vorname,nutzliznehm_strasse,nutzliznehm_ort,location,band,promoter,vg_state,vg_raum_clean,vg_raum_wo_stopwords,vg_raum_keywords_x,vg_datum_year,vg_datum_month,vg_datum_day_of_week,vg_datum_season,vg_raum_keywords_y,place_kirche,place_hotel,place_cafe,place_theater,place_club,place_halle,place_gaststaette,place_festhalle,place_kulturzentrum,place_festzelt,place_schloss,place_pub,place_stadthalle,place_park,place_gasthof,place_kabarett,place_arena,place_schlachthof,place_wandelhalle,place_turnhalle,place_buergerhaus,place_museum,place_rathaus,place_staatsbad,place_zelt,place_jazz,place_forum,place_gymnasium,place_schule,place_sporthalle,vg_state_y,state_bavaria,state_thuringia,state_baden-wuerttemberg,state_rhineland-palatinate,state_brandenburg,state_berlin,state_lower saxony,state_schleswig-holstein,state_hamburg,state_hesse,state_saarland,state_north rhine-westphalia,state_saxony-anhalt,state_saxony,state_mecklenburg-western pomerania,state_bremen,band_y,band_kurorchester bad wil,band_musikverein harmonie,band_kasalla,band_cat ballou,band_roncalli royal orch,band_jugendblasorchester,band_kurorchester bad pyr,band_hoehner,band_paveier,band_domstuermer,band_kluengelkoepp,band_alleinunterhalter,band_the gregorian voices,band_brings,band_musica hungarica,band_concerto,band_bad salzuflen orches,band_musikverein stadtkap,band_salonorchester hunga,band_miljoe,band_raeuber,band_kabarett leipziger f,band_marita koellner,band_salon-orchester hung,band_blaeck foeoess,band_schuelerinnen und sc,band_romain vicente,band_staatliche kurkapell,band_musikzug der freiwil,band_funky marys
ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
9938766,24643,2016,ABRE U INKA 2016,None,5027961235,4,NONRekla,2014-02-08,LISBERG,TURNHALLE SV RW LISBERG,20140208.0,TURNHALLE SV RW LISBERG,STAERK REINHOLD,DAS BOEHMISCHE FEUER,892,U-K (MUSIKER),18293091,210.00,NaN,NaN,1035530900,AURACHTALER BLASMUSIKVEREIN WALSDORF E.V,SCHINDHOLZWEG 17,96194,WALSDORF,1510423300,BAYERISCHER BLASMUSIKVERBAND,None,None,MUENCHEN,LISBERG TURNHALLE SV RW LISBERG,DAS BOEHMISCHE FEUER,AURACHTALER BLASMUSIKVEREIN WALSDORF E.V 96194,BAVARIA,TURNHALLE SV RW LISBERG,turnhalle sv rw lisberg,"[turnhalle, sv, rw, lisberg]",2014,2,5,1,"[turnhalle, sv, rw, lisberg]",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,BAVARIA,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,DAS BOEHMISCHE FEUER,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9939272,24643,2016,ABRE U INKA 2016,None,5053180236,3,NONRekla,2014-02-22,WIPPERDORF,STAATLICHE GRUNDSCHULE WIPPERDORF,20140222.0,STAATLICHE GRUNDSCHULE WIPPERDORF,PATSCHKE RAINER,KYFFHAEUSER TANZ BAN,1087,U-V II. 1 (+II 2),17923031,85.06,NaN,NaN,1374109400,WIPPERDORFER CARNEVAL CLUB E.V,STRASSE DER EINHEIT 6,99752,WIPPERDORF,1374109400,WIPPERDORFER CARNEVAL CLUB,None,STRASSE DER EINHEIT 6,WIPPERDORF,WIPPERDORF STAATLICHE GRUNDSCHULE WIPPERDORF,KYFFHAEUSER TANZ BAN,WIPPERDORFER CARNEVAL CLUB E.V 99752,THURINGIA,STAATLICHE GRUNDSCHULE WIPPERDORF,staatliche grundschule wipperdorf,"[grundschule, wipperdorf]",2014,2,5,1,"[grundschule, wipperdorf]",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,THURINGIA,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,KYFFHAEUSER TANZ BAN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9939279,24643,2016,ABRE U INKA 2016,None,5053345001,3,NONRekla,2014-02-18,DINGOLFING,BMW-WERK DINGOLFING,20140218.0,BMW-WERK DINGOLFING,JUNDMAYER,BERUFSFACHSCHULE,1087,U-V II. 1 (+II 2),17757433,90.20,NaN,NaN,2000026695,BAYERISCHE MOTORENWERKE AG,MENGKOFENER STR. 13,84130,DINGOLFING,2000026695,BAYERISCHE MOTORE

In [6]:
print(len(df.band.unique()))#317011 - unique bands
print(len(df.promoter.unique()))#176456 - unique promoter/organizer
print(len(df.vg_state.unique()))#16 - unique bundesland/states

317011
176456
16


In [ ]:
df.band

In [9]:
data_path_inkasso_stats_for_features =Path('C:/Saravana/Projects/Intellizenz/intellizenz-model-training/data/export_inkasso_stats_for_features_2016_2020.parquet.gzip')

data_path_vg_state = Path('C:/Saravana/Projects/Intellizenz/intellizenz-model-training/data/export_vg_stats_for_range_of_vginkasso_2016_2020.parquet.gzip')
data_path_band = Path('C:/Saravana/Projects/Intellizenz/intellizenz-model-training/data/export_band_stats_for_range_of_vginkasso_2016_2020.parquet.gzip')
data_path_promoter = Path('C:/Saravana/Projects/Intellizenz/intellizenz-model-training/data/export_promotor_stats_for_range_of_vginkasso_2016_2020.parquet.gzip')

# stat_var = ['location', 'band_x', 'promoter']
# stat_var = ['band']
stat_var = ['vg_state']
# stat_var = ['promoter']
# stat_var = ['band']
df_stat = {}



# 1. Take vg_inkasso value before the current datetime - (split the train and test and calculate the inkasso distribution-after feature calculation)
# 2. Timestamp as index, and have location/band as columns in df
# 3. Predefine the single df and iterate over it

def get_descr_stat(row):
    # row represents each line(row) in the dataframe
    def descr_stat(row, var):
        # Initialize a empty pandas dataframe
        # result = pd.Series([], dtype=pd.StringDtype())
        # result = pd.Series(dtype='float64')

        if not pd.isnull(row[var]):
            #get vg_inkasso collected for past events(that occured before current date of the row) for each location/state, each band and each promoter
            inkasso = df[(df['vg_datum_von']<row['vg_datum_von'])&(df[var] == row[var])]['vg_inkasso'].values.tolist()

            # Returns the mean, count, std, max, min and 0.05, 0.10..0.95 percentiles of the list of vg_inkasso values
            # for a specific attribute like location, band, promotor
            result = pd.Series(inkasso, dtype=pd.Float64Dtype()).describe(percentiles=percentiles)
            result = result.add_prefix('{}_'.format(var))

        result.name = row.name
        return result

    descr_stat_result = pd.Series(dtype='float64')
    # descr_stat_result = pd.Series([], dtype=pd.StringDtype())
    for v in stat_var:
        descr_stat_result = pd.concat([descr_stat_result, descr_stat(row, v)])

    return descr_stat_result


if data_path_inkasso_stats_for_features.exists() :
    df_featurize_descr_stat = pd.read_parquet(data_path_inkasso_stats_for_features)

elif data_path_vg_state.exists() and data_path_band.exists() and data_path_promoter.exists():
    df_state = pd.read_parquet(data_path_vg_state)
    df_band = pd.read_parquet(data_path_band)
    df_promoter = pd.read_parquet(data_path_promoter)

    df_featurize_descr_stat = df_state.merge(df_band, how='outer', on='ID').merge(df_promoter,
                                                                                how='outer',
                                                                                on='ID')
else:

    # for v in tqdm(stat_var):
    #     #group vg_inkasso collected for each location/state, each band and each promoter
    #     df_stat[v] = df.groupby(v)['vg_inkasso'].apply(list)

    percentiles = [round(x, 2) for x in np.linspace(0, 1, 21)[1:-1].tolist()]


    # for each_feature in stat_var:
    #     feature_index = stat_var.index(each_feature)

    #     feature = stat_var[feature_index]

    #     # Using DataFrame.copy() create new DataFrame.
    #     each_feature_df = df[[feature]].copy() #get specific feature in a new dataframe

    #     # Initialize a empty pandas dataframe of type float64
    #     result_df = pd.Series(dtype='float64') 

    #     # Add empty(np.nan) columns to new 'each_feature_df' dataframe
    #     col = 'count', 'mean', 'std', 'min','5%', '10%', '15%', '20%', '25%', '30%', '35%', '40%', '45%', 
    #     '50%', '55%', '60%', '65%','70%','75%','80%', '85%', '90%','95%', 'max'
    #     for each in col:
    #         cname = each_feature+'_'+each
    #         each_feature_df[cname] = np.nan
        
    #     # iterate over each rows in df
    #     for index, row in tqdm(df.iterrows()):
            
    #         match_condition = df[each_feature] == row[each_feature]
    #         filtered_df = df[match_condition] # Filter the dataframe with respect to feature 'each_feature' in iteration

    #         current_date = row.vg_datum_von
    #         prev_date_condition = filtered_df.vg_datum_von<current_date
    #         past_events_df = filtered_df[prev_date_condition] # get events that occured before current row event date
    #         inkasso = past_events_df['vg_inkasso'].values.tolist() # get all the past inkasso values for specific feature 'each_feature'

    #         result_df = pd.Series(inkasso).describe(percentiles=percentiles)

    #         for each in col:     
    #             cname = each_feature+'_'+each
    #             each_feature_df.at[index, cname] = result_df[each]
        
            
    #     df_featurize_descr_stat = each_feature_df


    # Each row of the dataframe 'df' is passed as row argument for get_descr_stat function
    df_featurize_descr_stat = df.progress_apply(get_descr_stat, axis=1)
    # df_featurize_descr_stat.to_pickle('./data/export_descr_stat_2016_2020.pkl.bz2', protocol=4)

display(df_featurize_descr_stat.head(5))

  0%|          | 0/1742241 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [48]:
def extract_features(features, target):
    data_path_inkasso_stats_for_features =Path('C:/Saravana/Projects/Intellizenz/intellizenz-model-training/data/export_inkasso_stats_for_features_2016_2020.parquet.gzip')

    data_path_vg_state = Path('C:/Saravana/Projects/Intellizenz/intellizenz-model-training/data/export_vg_stats_for_range_of_vginkasso_2016_2020.parquet.gzip')
    data_path_band = Path('C:/Saravana/Projects/Intellizenz/intellizenz-model-training/data/export_band_stats_for_range_of_vginkasso_2016_2020.parquet.gzip')
    data_path_promoter = Path('C:/Saravana/Projects/Intellizenz/intellizenz-model-training/data/export_promotor_stats_for_range_of_vginkasso_2016_2020.parquet.gzip')

    # stat_var = ['location', 'band_x', 'promoter']
    # stat_var = ['band']
    # stat_var = ['vg_state']
    stat_var = ['promoter']
    # stat_var = ['band']
    df_stat = {}


    # 1. Take vg_inkasso value before the current datetime - (split the train and test and calculate the inkasso distribution-after feature calculation)
    # 2. Timestamp as index, and have location/band as columns in df
    # 3. Predefine the single df and iterate over it

    def get_descr_stat(row):
        # row represents each line(row) in the dataframe
        def descr_stat(row, var):
            # Initialize a empty pandas dataframe
            result = pd.Series(dtype='float64')

            if not pd.isnull(row[var]):
                #get vg_inkasso collected for past events(that occured before current date of the row) for each location/state, each band and each promoter
                matching_indexes = features[(features['vg_datum_von']<row['vg_datum_von'])&(features[var] == row[var])].index.values.tolist()

                inkasso =  list(target[target.index.isin(matching_indexes)].values)

                # Returns the mean, count, std, max, min and 0.05, 0.10..0.95 percentiles of the list of vg_inkasso values
                # for a specific attribute like location, band, promotor
                result = pd.Series(inkasso, dtype='float64').describe(percentiles=percentiles)
                result = result.add_prefix('{}_'.format(var))

            result.name = row.name
            return result

        descr_stat_result = pd.Series(dtype='float64')
        for v in stat_var:
            descr_stat_result = pd.concat([descr_stat_result, descr_stat(row, v)])

        return descr_stat_result

 
    if data_path_inkasso_stats_for_features.exists() :
        df_featurize_descr_stat = pd.read_parquet(data_path_inkasso_stats_for_features)

    elif data_path_vg_state.exists() and data_path_band.exists() and data_path_promoter.exists():
        df_state = pd.read_parquet(data_path_vg_state)
        df_band = pd.read_parquet(data_path_band)
        df_promoter = pd.read_parquet(data_path_promoter)

        df_featurize_descr_stat = df_state.merge(df_band, how='outer', on='ID').merge(df_promoter,
                                                                                    how='outer',
                                                                                    on='ID')
    else:
        percentiles = [round(x, 2) for x in np.linspace(0, 1, 21)[1:-1].tolist()]

        # Each row of the dataframe 'df' is passed as row argument for get_descr_stat function
        df_featurize_descr_stat = features.progress_apply(get_descr_stat, axis=1)
        # df_featurize_descr_stat.to_pickle('./data/export_descr_stat_2016_2020.pkl.bz2', protocol=4)

    display(df_featurize_descr_stat.head(5))
    return df_featurize_descr_stat

df_featurize_descr_stat = extract_features(X_test, y_test)

  0%|          | 0/348449 [00:00<?, ?it/s]

,promoter_10%,promoter_15%,promoter_20%,promoter_25%,promoter_30%,promoter_35%,promoter_40%,promoter_45%,promoter_5%,promoter_50%,promoter_55%,promoter_60%,promoter_65%,promoter_70%,promoter_75%,promoter_80%,promoter_85%,promoter_90%,promoter_95%,promoter_count,promoter_max,promoter_mean,promoter_min,promoter_std
ID,,,,,,,,,,,,,,,,,,,,,,,,
13331491,78.250,78.2500,78.250,78.2500,78.250,78.2500,78.250,78.2500,78.2500,78.250,78.2500,78.250,78.2500,78.250,78.2500,78.250,78.2500,78.250,78.2500,1.0,78.25,78.250000,78.25,NaN
10619045,18.240,18.2400,18.240,18.2400,18.240,18.2400,18.240,18.2400,18.2400,18.240,18.2400,18.240,18.2400,18.240,18.2400,18.240,18.2400,18.240,18.2400,1.0,18.24,18.240000,18.24,NaN
11723390,20503.140,20503.1400,20503.140,20503.1400,20503.140,20503.1400,20503.140,20503.1400,13430.2945,20503.140,20503.1400,20503.140,20503.1400,20503.140,20503.1400,20503.140,20503.1400,20503.140,20503.1400,14.0,20503.14,19059.702143,295.01,5400.849921
10809092,60.773,63.7995,66.826,69.8525,72.879,75.9055,78.932,81.9585,57.7465,84.985,88.0115,91.038,94.0645,97.091,100.1175,103.144,106.1705,109.197,112.2235,2.0,115.25,84.985000,54.72,42.801173
13349736,37.460,37.4600,37.460,37.4600,40.244,43.0280,45.812,48.5960,37.4600,51.380,58.3760,65.372,72.3680,79.364,86.3600,104.692,123.0240,141.356,159.6880,5.0,178.02,78.136000,37.46,59.307262


In [43]:
df_featurize_descr_stat.to_parquet('C:/Saravana/Projects/Intellizenz/intellizenz-model-training/data/export_train_band_stats_for_range_of_vginkasso_2016_2020.parquet.gzip',compression='gzip')

In [45]:
df_featurize_descr_stat.to_parquet('C:/Saravana/Projects/Intellizenz/intellizenz-model-training/data/export_test_band_stats_for_range_of_vginkasso_2016_2020.parquet.gzip',compression='gzip')

In [47]:
df_featurize_descr_stat.to_parquet('C:/Saravana/Projects/Intellizenz/intellizenz-model-training/data/export_test_vg_state_stats_for_range_of_vginkasso_2016_2020.parquet.gzip',compression='gzip')

In [49]:
df_featurize_descr_stat.to_parquet('C:/Saravana/Projects/Intellizenz/intellizenz-model-training/data/export_test_promoter_stats_for_range_of_vginkasso_2016_2020.parquet.gzip',compression='gzip')

In [ ]:
# Without using progress apply and using For-loop to extract features 

data_path_inkasso_stats_for_features =Path('C:/Saravana/Projects/Intellizenz/intellizenz-model-training/data/export_inkasso_stats_for_features_2016_2020.parquet.gzip')

data_path_vg_state = Path('C:/Saravana/Projects/Intellizenz/intellizenz-model-training/data/export_vg_stats_for_range_of_vginkasso_2016_2020.parquet.gzip')
data_path_band = Path('C:/Saravana/Projects/Intellizenz/intellizenz-model-training/data/export_band_stats_for_range_of_vginkasso_2016_2020.parquet.gzip')
data_path_promoter = Path('C:/Saravana/Projects/Intellizenz/intellizenz-model-training/data/export_promotor_stats_for_range_of_vginkasso_2016_2020.parquet.gzip')

# stat_var = ['location', 'band_x', 'promoter']
# stat_var = ['band']
stat_var = ['vg_state']
# stat_var = ['promoter']
# stat_var = ['band']
df_stat = {}



# 1. Take vg_inkasso value before the current datetime - (split the train and test and calculate the inkasso distribution-after feature calculation)
# 2. Timestamp as index, and have location/band as columns in df
# 3. Predefine the single df and iterate over it

feature_index = 0


if data_path_inkasso_stats_for_features.exists() :
    df_featurize_descr_stat = pd.read_parquet(data_path_inkasso_stats_for_features)

elif data_path_vg_state.exists() and data_path_band.exists() and data_path_promoter.exists():
    df_state = pd.read_parquet(data_path_vg_state)
    df_band = pd.read_parquet(data_path_band)
    df_promoter = pd.read_parquet(data_path_promoter)

    df_featurize_descr_stat = df_state.merge(df_band, how='outer', on='ID').merge(df_promoter,
                                                                                how='outer',
                                                                                on='ID')
else:

    percentiles = [round(x, 2) for x in np.linspace(0, 1, 21)[1:-1].tolist()]


    for each_feature in stat_var:
        feature_index = stat_var.index(each_feature)

        feature = stat_var[feature_index]

        # Using DataFrame.copy() create new DataFrame.
        each_feature_df = df[[feature]].copy() #get specific feature in a new dataframe

        # Initialize a empty pandas dataframe of type float64
        result_df = pd.Series(dtype='float64') 

        # Add empty(np.nan) columns to new 'each_feature_df' dataframe
        col = 'count', 'mean', 'std', 'min','5%', '10%', '15%', '20%', '25%', '30%', '35%', '40%', '45%', 
        '50%', '55%', '60%', '65%','70%','75%','80%', '85%', '90%','95%', 'max'
        for each in col:
            cname = each_feature+'_'+each
            each_feature_df[cname] = np.nan
        
        # iterate over each rows in df
        for index, row in tqdm(df.iterrows()):
            
            match_condition = df[each_feature] == row[each_feature]
            filtered_df = df[match_condition] # Filter the dataframe with respect to feature 'each_feature' in iteration

            current_date = row.vg_datum_von
            prev_date_condition = filtered_df.vg_datum_von<current_date
            past_events_df = filtered_df[prev_date_condition] # get events that occured before current row event date
            inkasso = past_events_df['vg_inkasso'].values.tolist() # get all the past inkasso values for specific feature 'each_feature'

            result_df = pd.Series(inkasso).describe(percentiles=percentiles)

            for each in col:     
                cname = each_feature+'_'+each
                each_feature_df.at[index, cname] = result_df[each]
        
            
        df_featurize_descr_stat = each_feature_df

display(df_featurize_descr_stat.head(5))

In [ ]:
df_featurize_descr_stat.to_parquet('C:/Saravana/Projects/Intellizenz/intellizenz-model-training/data/export_band_stats_for_range_of_vginkasso_2016_2020.parquet.gzip',compression='gzip')

In [ ]:
df_featurize_descr_stat.to_parquet('C:/Saravana/Projects/Intellizenz/intellizenz-model-training/data/export_promotor_stats_for_range_of_vginkasso_2016_2020.parquet.gzip',compression='gzip')

In [ ]:
df_featurize_descr_stat.to_parquet('C:/Saravana/Projects/Intellizenz/intellizenz-model-training/data/export_vg_stats_for_range_of_vginkasso_2016_2020.parquet.gzip',compression='gzip')

In [ ]:
# When the time varied vg_inkasso distribution are extracted for vg_state, band, promoter, merge these features
if data_path_vg_state.exists() and data_path_band.exists() and data_path_promoter.exists():
    df_state = pd.read_parquet(data_path_vg_state)
    df_band = pd.read_parquet(data_path_band)
    df_promoter = pd.read_parquet(data_path_promoter)

    df_featurize_descr_stat = df_state.merge(df_band, how='outer', on='ID').merge(df_promoter,
                                                                                how='outer',
                                                                                on='ID')
if not data_path_inkasso_stats_for_features.exists() :
    df_featurize_descr_stat.to_parquet('C:/Saravana/Projects/Intellizenz/intellizenz-model-training/data/export_inkasso_stats_for_features_2016_2020.parquet.gzip',compression='gzip')